# Perceptron (Single Layer Perceptron)

perceptron 은 인공신경망의 종류중 하나입니다.

Neuron을 본따 만듦

초창기 Perceptron은 Linear classifier 를 위해 만들어진 모델입니다. 

어떤 두가지 클래스가 있을때 그것을 linear하게 나눌수 있는 분류기를 말합니다. 

(실제로는 AND, OR 문제를 해결하기 위해만들어졌다고 합니다.)

In [5]:
# AND와 OR
print(0 and 0, 0 or 0)
print(0 and 1, 1 or 1)
print(1 and 0, 1 or 0)
print(1 and 1 ,1 or 1)

0 0
0 1
0 1
1 1


위 and or 문제는 perceptron 으로 해결이 쉽게 가능하지만,  
minski 는 한개의 레이어 구조를 가지고 있는 Perceptron 구조로는 XOR문제는 해결할 수 없다라는 것을 증명 하였습니다.  
그래서 Multilayer를 이용해야지만 이 것을 해결할 수 있다 하지만 Multilayer는 각층의 W들을 학습시킬수 없다라고 말하며, 인공신경망의 암흑기가 옵니다.  

![XOR형태](http://www.saedsayad.com/images/Perceptron_XOR.png)
딱봐도 Linear한 한직선으로는 이것을 분류할 수 없어 보입니다.


하지만 우리모두가 알고 있듯이 Backpropagtaion을 사용하면서, 지금은 문제가 해결 되었습니다. 

![](https://www.cse.unsw.edu.au/~cs9417ml/MLP2/BPNeuralNetwork.jpg)




# XOR 코드 구현

In [2]:
import torch

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
X = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
Y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

# nn Layer
linear = torch.nn.Linear(2,1, bias=True) 
# Perceptron 같은 경우 한개의 Fully Connected Layer를 갖는구조
# 일단 처음엔 Single Layer
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear, sigmoid).to(device)

# define cost/loss $ optimizer
criterion = torch.nn.BCELoss().to(device) # X값이 0,1과 1을 분리하는 Binary Classification Task 이므로 Loss로는 Binary Cross Entropy Loss를 사용합니다.
optimizer = torch.optim.SGD(model.parameters(), lr=1)
# Optimizer로는 이전에도 이용하는 statistic gradient descent 를 사용

for step in range(1001) :
    optimizer.zero_grad()
    hypothesis = model(X)
    
    # cost/loss function
    cost = criterion(hypothesis, Y)
    # model이 X,Y 게이트와 같은 값이 나올수 있도록 학습
    cost.backward()
    optimizer.step()
    if step % 100 == 0 :
        print(step, cost.item())

0 0.7273974418640137
100 0.6931475400924683
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996
1000 0.6931471824645996


In [5]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[0.5]
 [0.5]
 [0.5]
 [0.5]] 
Correct:  [[0.]
 [0.]
 [0.]
 [0.]] 
Accuracy:  0.5


200 번째 step부턴 계속 같은 cost를 뱉어내는것을 볼수 있습니다.
이걸 해결하기 위해 우리는 Multi Layer Peceptron을 배울 예정입니다.